In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import time
import logging


In [2]:
#!pip install mlxtend  

In [3]:
pd.set_option('display.max_columns',None)

dataset = [['Apple', 'Beer', 'Rice', 'Chicken'],  ['Apple', 'Beer', 'Rice'], ['Apple', 'Beer'],  ['Apple', 'Bananas'], ['Milk', 'Beer', 'Rice', 'Chicken'], ['Milk', 'Beer', 'Rice'],  ['Milk', 'Beer'], ['Apple', 'Bananas']]

dataset

[['Apple', 'Beer', 'Rice', 'Chicken'],
 ['Apple', 'Beer', 'Rice'],
 ['Apple', 'Beer'],
 ['Apple', 'Bananas'],
 ['Milk', 'Beer', 'Rice', 'Chicken'],
 ['Milk', 'Beer', 'Rice'],
 ['Milk', 'Beer'],
 ['Apple', 'Bananas']]

In [4]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
#arr=te_ary.astype(np.int)
#arr
te_ary

array([[ True, False,  True,  True, False,  True],
       [ True, False,  True, False, False,  True],
       [ True, False,  True, False, False, False],
       [ True,  True, False, False, False, False],
       [False, False,  True,  True,  True,  True],
       [False, False,  True, False,  True,  True],
       [False, False,  True, False,  True, False],
       [ True,  True, False, False, False, False]])

In [5]:
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Apple,Bananas,Beer,Chicken,Milk,Rice
0,True,False,True,True,False,True
1,True,False,True,False,False,True
2,True,False,True,False,False,False
3,True,True,False,False,False,False
4,False,False,True,True,True,True
5,False,False,True,False,True,True
6,False,False,True,False,True,False
7,True,True,False,False,False,False


In [6]:
support_threshold = 0.2

frequent_itemsets = apriori(df, min_support= support_threshold, use_colnames = True)

In [10]:
frequent_itemsets

,support,itemsets
0,0.625,(Apple)
1,0.250,(Bananas)
2,0.750,(Beer)
3,0.250,(Chicken)
4,0.375,(Milk)
5,0.500,(Rice)
6,0.250,"(Bananas, Apple)"
7,0.375,"(Beer, Apple)"
8,0.250,"(Rice, Apple)"
9,0.250,"(Beer, Chicken)"


In [11]:
supportRules1 = association_rules(frequent_itemsets, metric="support", min_threshold = 0.2)
print(supportRules1)

        antecedents      consequents  antecedent support  consequent support  \
0         (Bananas)          (Apple)               0.250               0.625   
1           (Apple)        (Bananas)               0.625               0.250   
2            (Beer)          (Apple)               0.750               0.625   
3           (Apple)           (Beer)               0.625               0.750   
4            (Rice)          (Apple)               0.500               0.625   
5           (Apple)           (Rice)               0.625               0.500   
6            (Beer)        (Chicken)               0.750               0.250   
7         (Chicken)           (Beer)               0.250               0.750   
8            (Beer)           (Milk)               0.750               0.375   
9            (Milk)           (Beer)               0.375               0.750   
10           (Beer)           (Rice)               0.750               0.500   
11           (Rice)           (Beer)    

In [12]:
print(supportRules1[['antecedents', 'consequents', 'support','confidence','lift']])

        antecedents      consequents  support  confidence      lift
0         (Bananas)          (Apple)    0.250    1.000000  1.600000
1           (Apple)        (Bananas)    0.250    0.400000  1.600000
2            (Beer)          (Apple)    0.375    0.500000  0.800000
3           (Apple)           (Beer)    0.375    0.600000  0.800000
4            (Rice)          (Apple)    0.250    0.500000  0.800000
5           (Apple)           (Rice)    0.250    0.400000  0.800000
6            (Beer)        (Chicken)    0.250    0.333333  1.333333
7         (Chicken)           (Beer)    0.250    1.000000  1.333333
8            (Beer)           (Milk)    0.375    0.500000  1.333333
9            (Milk)           (Beer)    0.375    1.000000  1.333333
10           (Beer)           (Rice)    0.500    0.666667  1.333333
11           (Rice)           (Beer)    0.500    1.000000  1.333333
12           (Rice)        (Chicken)    0.250    0.500000  2.000000
13        (Chicken)           (Rice)    0.250   

In [13]:
lift1 = association_rules(frequent_itemsets, metric="lift", min_threshold=1.5)
print(lift1)
lift1
print(lift1[['antecedents', 'consequents', 'support', 'lift','confidence']])

       antecedents      consequents  antecedent support  consequent support  \
0        (Bananas)          (Apple)               0.250               0.625   
1          (Apple)        (Bananas)               0.625               0.250   
2           (Rice)        (Chicken)               0.500               0.250   
3        (Chicken)           (Rice)               0.250               0.500   
4     (Beer, Rice)        (Chicken)               0.500               0.250   
5  (Beer, Chicken)           (Rice)               0.250               0.500   
6           (Rice)  (Beer, Chicken)               0.500               0.250   
7        (Chicken)     (Beer, Rice)               0.250               0.500   

   support  confidence  lift  leverage  conviction  
0     0.25         1.0   1.6   0.09375         inf  
1     0.25         0.4   1.6   0.09375        1.25  
2     0.25         0.5   2.0   0.12500        1.50  
3     0.25         1.0   2.0   0.12500         inf  
4     0.25         0.5 

In [14]:
lift1[(lift1.confidence > .5)  & (lift1.support > .2)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bananas),(Apple),0.25,0.625,0.25,1.0,1.6,0.09375,inf
3,(Chicken),(Rice),0.25,0.500,0.25,1.0,2.0,0.12500,inf
5,"(Beer, Chicken)",(Rice),0.25,0.500,0.25,1.0,2.0,0.12500,inf
7,(Chicken),"(Beer, Rice)",0.25,0.500,0.25,1.0,2.0,0.12500,inf


In [15]:
confidence1 = association_rules(frequent_itemsets, metric="confidence", min_threshold=1)
print(confidence1)

print(confidence1[['antecedents', 'consequents', 'support','confidence']])


       antecedents   consequents  antecedent support  consequent support  \
0        (Bananas)       (Apple)               0.250               0.625   
1        (Chicken)        (Beer)               0.250               0.750   
2           (Milk)        (Beer)               0.375               0.750   
3           (Rice)        (Beer)               0.500               0.750   
4        (Chicken)        (Rice)               0.250               0.500   
5    (Rice, Apple)        (Beer)               0.250               0.750   
6  (Beer, Chicken)        (Rice)               0.250               0.500   
7  (Rice, Chicken)        (Beer)               0.250               0.750   
8        (Chicken)  (Beer, Rice)               0.250               0.500   
9     (Milk, Rice)        (Beer)               0.250               0.750   

   support  confidence      lift  leverage  conviction  
0    0.250         1.0  1.600000   0.09375         inf  
1    0.250         1.0  1.333333   0.06250       